In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv('traffic_stops - traffic_stops_with_vehicle_number.csv')

# Preview the data
df.head()
df.info()
df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   stop_date           65538 non-null  object
 1   stop_time           65538 non-null  object
 2   country_name        65538 non-null  object
 3   driver_gender       65538 non-null  object
 4   driver_age_raw      65538 non-null  int64 
 5   driver_age          65538 non-null  int64 
 6   driver_race         65538 non-null  object
 7   violation_raw       65538 non-null  object
 8   violation           65538 non-null  object
 9   search_conducted    65538 non-null  bool  
 10  search_type         43818 non-null  object
 11  stop_outcome        65538 non-null  object
 12  is_arrested         65538 non-null  bool  
 13  stop_duration       65538 non-null  object
 14  drugs_related_stop  65538 non-null  bool  
 15  vehicle_number      65538 non-null  object
dtypes: bool(3), int64(2), 

C:\Users\LOQ\AppData\Local\Temp\ipykernel_19432\3501991639.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('traffic_stops - traffic_stops_with_vehicle_number.csv')


,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
count,65538,65538,65538,65538,65538.000000,65538.00000,65538,65538,65538,65538,43818,65538,65538,65538,65538,65538
unique,46,1440,3,2,NaN,NaN,5,5,5,2,2,3,2,3,2,65538
top,2020-01-01,0:00:00,India,F,NaN,NaN,White,Seatbelt,Other,False,Frisk,Warning,True,30+ Min,True,UP76DY3473
freq,1440,46,21998,32881,NaN,NaN,13168,13204,13194,32837,21971,21966,32846,21958,32769,1
mean,NaN,NaN,NaN,NaN,49.055998,49.11221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,18.174699,18.15012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,18.000000,18.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,33.000000,34.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,49.000000,49.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,65.000000,65.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns


Index(['stop_date', 'stop_time', 'country_name', 'driver_gender',
       'driver_age_raw', 'driver_age', 'driver_race', 'violation_raw',
       'violation', 'search_conducted', 'search_type', 'stop_outcome',
       'is_arrested', 'stop_duration', 'drugs_related_stop', 'vehicle_number'],
      dtype='object')

In [5]:
df.isnull().sum()

stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64

In [7]:
# Drop all-null columns
df = df.dropna(axis=1, how='all')

In [6]:
df = df.fillna({
    'country_name': 'Unknown',
    'driver_gender': 'Unknown',
    'driver_race': 'Unknown',
    'violation': 'Unknown',
    'search_type': 'None',
    'stop_outcome': 'Unknown',
    'stop_duration': 'Unknown',
    'vehicle_number': 'Unknown'
})
df = df.fillna(0)

In [9]:
# Convert to correct datatypes
# Safely convert driver_age to numeric, fill missing with the median, then cast to int
drivers_age = pd.to_numeric(df['driver_age'], errors='coerce')
df['driver_age'] = drivers_age.fillna(drivers_age.median()).astype(int)

# Ensure boolean columns are boolean; fill missing with False first
for col in ['search_conducted', 'is_arrested', 'drugs_related_stop']:
	df[col] = df[col].fillna(False).astype(bool)

In [10]:
df['timestamp'] = pd.to_datetime(df['stop_date'] + ' ' + df['stop_time'], errors='coerce')
df = df.dropna(subset=['timestamp'])

In [18]:
columns_needed = [
    'stop_date', 'stop_time','country_name', 'driver_gender', 'driver_age',
    'driver_race', 'violation', 'search_conducted', 'search_type',
    'stop_outcome', 'is_arrested', 'stop_duration',
    'drugs_related_stop', 'vehicle_number','timestamp'
]
df = df[columns_needed]

In [19]:
df.to_csv("traffic_stops_clean.csv", index=False)

In [20]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [21]:
#connect to pgadmin
connection = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password = "moon123",
    port = 5432
)
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
mediator = connection.cursor()
mediator.execute("select version()")
version = mediator.fetchone()
print(f"the database version is: {version}")

the database version is: ('PostgreSQL 15.12, compiled by Visual C++ build 1943, 64-bit',)


In [22]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
#connect to pgadmin and create a database
connection = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password = "moon123",
    port = 5432,  
    database = "securecheck"
)

connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
writer = connection.cursor()
#check if the connection is successful
writer.execute("SELECT version()")
version = writer.fetchone() 
print(f"Connected to the database: {version[0]}")

#creating a table in the database

while True:
    try:
        writer.execute("DROP TABLE IF EXISTS police_log")
        break
    except psycopg2.Error as e:
        print(f"Error dropping table: {e}")
        break

#creating a table in the database

writer.execute("""
CREATE TABLE IF NOT EXISTS police_log (
    id SERIAL PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    is_arrested BOOLEAN,    
    stop_duration VARCHAR(50),
    drugs_related_stop BOOLEAN,
    vehicle_number VARCHAR(50),
    timestamp TIMESTAMP
)
""")

#check if the table is created successfully
writer.execute("SELECT * FROM police_log LIMIT 1")
if writer.rowcount == 0:
    print("Table 'police_log' created successfully.")
else:
    print("Table 'police_log' already exists.")
# Data insertion into the table
# Ensure the DataFrame has the correct columns

Connected to the database: PostgreSQL 15.12, compiled by Visual C++ build 1943, 64-bit
Table 'police_log' created successfully.


In [23]:
# insert query
insert_query = """
INSERT INTO police_log 
(stop_date, stop_time, country_name, driver_gender, driver_age, driver_race, violation,
search_conducted, search_type, stop_outcome, is_arrested,stop_duration, drugs_related_stop, vehicle_number, timestamp)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""" 
# iterate over the DataFrame and insert each row into the database
for index, row in df.iterrows():
    writer.execute(
        insert_query, (
            row['stop_date'],
            row['stop_time'],            
            row.get('country_name'),
            row.get('driver_gender'),
            int(row.get('driver_age')) if row.get('driver_age') is not None else None,
            row.get('driver_race'),
            row.get('violation'),
            bool(row.get('search_conducted')) if row.get('search_conducted') is not None else False,
            row.get('search_type'),
            row.get('stop_outcome'),
            bool(row.get('is_arrested')) if row.get('is_arrested') is not None else False,
            row.get('stop_duration'),
            bool(row.get('drugs_related_stop')) if row.get('drugs_related_stop') is not None else False,            
            row.get('vehicle_number', f'VEH{row.name}'),
            row.get('timestamp')
        )
    )

# commit the changes    
connection.commit()
# close the connection
writer.close()
connection.close()
# Display the cleaned DataFrame
print("Data inserted successfully into the police_log table.")


Data inserted successfully into the police_log table.
